<a href="https://colab.research.google.com/github/joseogg/textanalytics/blob/main/Clasificacion_texto_ejercicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación de Texto

## Librerías

In [ ]:
!python -m pip install contractions

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import re
import spacy
import contractions
from sklearn.datasets import fetch_20newsgroups
import warnings

warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Extracción de datos sobre noticias

In [ ]:
data = fetch_20newsgroups(subset="all", shuffle=True, remove=("headers", "footers", "quotes"))
mapa_etiquetas = dict(enumerate(data.target_names))
mapa_etiquetas

{0: 'alt.atheism',
 1: 'comp.graphics',
 2: 'comp.os.ms-windows.misc',
 3: 'comp.sys.ibm.pc.hardware',
 4: 'comp.sys.mac.hardware',
 5: 'comp.windows.x',
 6: 'misc.forsale',
 7: 'rec.autos',
 8: 'rec.motorcycles',
 9: 'rec.sport.baseball',
 10: 'rec.sport.hockey',
 11: 'sci.crypt',
 12: 'sci.electronics',
 13: 'sci.med',
 14: 'sci.space',
 15: 'soc.religion.christian',
 16: 'talk.politics.guns',
 17: 'talk.politics.mideast',
 18: 'talk.politics.misc',
 19: 'talk.religion.misc'}

## Creación de DataFrame

In [ ]:
corpus, etiqueta_nums, etiqueta_noms = (data.data, data.target, 
                                       [mapa_etiquetas[etiqueta] for etiqueta in data.target])

datos = pd.DataFrame({"articulo": corpus, "etiqueta_num": etiqueta_nums, "etiqueta_nom": etiqueta_noms})

print(datos.shape)
datos.head(10)

(18846, 3)


,articulo,etiqueta_num,etiqueta_nom
0,\n\nI am sure some bashers of Pens fans are pr...,10,rec.sport.hockey
1,My brother is in the market for a high-perform...,3,comp.sys.ibm.pc.hardware
2,\n\n\n\n\tFinally you said what you dream abou...,17,talk.politics.mideast
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3,comp.sys.ibm.pc.hardware
4,1) I have an old Jasmine drive which I cann...,4,comp.sys.mac.hardware
5,\n\nBack in high school I worked as a lab assi...,12,sci.electronics
6,\n\nAE is in Dallas...try 214/241-6060 or 214/...,4,comp.sys.mac.hardware
7,"\n[stuff deleted]\n\nOk, here's the solution t...",10,rec.sport.hockey
8,"\n\n\nYeah, it's the second one. And I believ...",10,rec.sport.hockey
9,\nIf a Christian means someone who believes in...,19,talk.religion.misc


# Quitando noticias nulas

In [ ]:
total_nulos = datos[datos.articulo.str.strip() == ""].shape[0]
print("Documentos vacíos:", total_nulos)

Documentos vacíos: 515


In [ ]:
datos = datos[~(datos.articulo.str.strip() == "")]
datos.shape

(18331, 3)

## Normalización de las noticias:
### Contracciones, Stopwords, Lematización

In [ ]:
!python -m spacy download en_core_web_sm   
#!python -m spacy download es_core_news_sm  
nltk.download("stopwords")

nlp = spacy.load("en_core_web_sm")
stopwords = nltk.corpus.stopwords.words("english")
df = pd.DataFrame(columns=["articulo", "articulo_limpio", "etiqueta_num", "etiqueta_nom"])
df["etiqueta_num"] = df["etiqueta_num"].astype("int64")
for registro in datos.values:
  registro = {"articulo": registro[0], "articulo_limpio": "", "etiqueta_num": registro[1], "etiqueta_nom": registro[2]}
  articulo = registro["articulo"] # toma solo el articulo

  articulo_expandido = ""
  for palabra in articulo.split():
    articulo_expandido += contractions.fix(palabra) + " "  
  articulo = articulo_expandido.strip().lower()
  articulo = articulo.lower()
  articulo = re.sub("[^A-Za-z]+", " ", articulo)
  articulo_limpio = ""
  for palabra in articulo.split():
    if palabra not in stopwords:
        articulo_limpio += palabra + " "
  
  documento = nlp(articulo_limpio.strip(), disable=["ner", "parser", "textcat"])
  articulo_lematizado = " "
  for palabra in documento:
    articulo_lematizado += palabra.lemma_ + " "
  if articulo_lematizado.strip() != "":
    registro["articulo_limpio"] = articulo_lematizado
    df = df.append(registro, ignore_index = True)

datos = df
datos

2022-10-12 17:20:05.282767: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,articulo,articulo_limpio,etiqueta_num,etiqueta_nom
0,\n\nI am sure some bashers of Pens fans are pr...,sure basher pen fan pretty confused lack kind...,10,rec.sport.hockey
1,My brother is in the market for a high-perform...,brother market high performance video card su...,3,comp.sys.ibm.pc.hardware
2,\n\n\n\n\tFinally you said what you dream abou...,finally say dream mediterranean new area grea...,17,talk.politics.mideast
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,think scsi card dma transfer disk scsi card d...,3,comp.sys.ibm.pc.hardware
4,1) I have an old Jasmine drive which I cann...,old jasmine drive can not use new system unde...,4,comp.sys.mac.hardware
...,...,...,...,...
18297,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,dn nyeda cnsvax uwec edu david nye dn neurolo...,13,sci.med
18298,\nNot in isolated ground recepticles (usually ...,isolated ground recepticle usually unusual co...,12,sci.electronics
18299,I just installed a DX2-66 CPU in a clone mothe...,instal dx cpu clone motherboard try mount cpu...,3,comp.sys.ibm.pc.hardware
18300,\nWouldn't this require a hyper-sphere. In 3-...,would require hyper sphere space point specif...,1,comp.graphics


In [ ]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18302 entries, 0 to 18301
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   articulo         18302 non-null  object
 1   articulo_limpio  18302 non-null  object
 2   etiqueta_num     18302 non-null  int64 
 3   etiqueta_nom     18302 non-null  object
dtypes: int64(1), object(3)
memory usage: 572.1+ KB


# Guardar datos normalizados

In [ ]:
datos.to_csv("newsgroups_limpio.csv", index=False)

In [ ]:
datos = pd.read_csv("newsgroups_limpio.csv")

In [ ]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18302 entries, 0 to 18301
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   articulo         18302 non-null  object
 1   articulo_limpio  18302 non-null  object
 2   etiqueta_num     18302 non-null  int64 
 3   etiqueta_nom     18302 non-null  object
dtypes: int64(1), object(3)
memory usage: 572.1+ KB


# Creación de datos de entrenamiento y prueba

In [ ]:
from sklearn.model_selection import train_test_split

train_corpus, test_corpus, train_etiqueta_nums, test_etiqueta_nums, train_etiqueta_noms, test_etiqueta_noms =\
                                 train_test_split(np.array(datos["articulo_limpio"]), np.array(datos["etiqueta_num"]),
                                                  np.array(datos["etiqueta_nom"]), test_size=0.33, random_state=1)

train_corpus.shape, test_corpus.shape

((12262,), (6040,))

# Contar datos de entrenamiento y prueba para cada clase

In [ ]:
from collections import Counter

trd = dict(Counter(train_etiqueta_noms))
tsd = dict(Counter(test_etiqueta_noms))

(pd.DataFrame([[key, trd[key], tsd[key]] for key in trd], 
             columns=["etiqueta_num", "conteo_entrenamiento", "conteo_prueba"])
.sort_values(by=["conteo_entrenamiento", "conteo_prueba"],
             ascending=False))

,etiqueta_num,conteo_entrenamiento,conteo_prueba
12,rec.motorcycles,672,297
10,comp.windows.x,666,314
6,sci.crypt,652,310
5,sci.med,650,310
9,rec.sport.baseball,650,302
19,rec.autos,645,289
11,comp.sys.ibm.pc.hardware,643,320
8,comp.os.ms-windows.misc,636,310
16,soc.religion.christian,633,341
3,misc.forsale,633,326


# Modelo de Bolsa de Palabras

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score

# Construye Bag-of-Words a partir de los datos de entrenamiento
cv = CountVectorizer(binary=False, min_df=0.0, max_df=1.0)
cv_train_features = cv.fit_transform(train_corpus)
# Construye Bag-of-Words a partir de los datos de prueba
cv_test_features = cv.transform(test_corpus)
print("Modelo BOW:> Entrenamiento:", cv_train_features.shape, " Prueba:", cv_test_features.shape)

Modelo BOW:> Entrenamiento: (12262, 64183)  Prueba: (6040, 64183)


# Bosque Aleatorio con Bolsa de Palabras

In [ ]:
from sklearn.ensemble import RandomForestClassifier

bosque = RandomForestClassifier(n_estimators=50, random_state=1)
bosque.fit(cv_train_features, train_etiqueta_noms)
bosque_bolsa_cv_scores = cross_val_score(bosque, cv_train_features, train_etiqueta_noms, cv=5)
bosque_bolsa_cv_mean_score = np.mean(bosque_bolsa_cv_scores)
print("CV Accuracy (5-fold):", bosque_bolsa_cv_scores)
print("Promedio CV Accuracy:", bosque_bolsa_cv_mean_score)
bosque_bolsa_test_score = bosque.score(cv_test_features, test_etiqueta_noms)
print("Test Accuracy:", bosque_bolsa_test_score)

CV Accuracy (5-fold): [0.64573991 0.64370159 0.62846656 0.637031   0.64926591]
Promedio CV Accuracy: 0.6408409917210274
Test Accuracy: 0.6627483443708609


# Regresión Logística con Bolsa de Palabras

In [ ]:
from sklearn.linear_model import LogisticRegression

regresion_log = LogisticRegression(max_iter=50, random_state=1)
regresion_log.fit(cv_train_features, train_etiqueta_noms)
regresion_log_bolsa_cv_scores = cross_val_score(regresion_log, cv_train_features, train_etiqueta_noms, cv=5)
regresion_log_bolsa_cv_mean_score = np.mean(regresion_log_bolsa_cv_scores)
print("CV Accuracy (5-fold):", regresion_log_bolsa_cv_scores)
print("Promedio CV Accuracy:", regresion_log_bolsa_cv_mean_score)
regresion_log_bolsa_test_score = regresion_log.score(cv_test_features, test_etiqueta_noms)
print("Test Accuracy:", regresion_log_bolsa_test_score)

CV Accuracy (5-fold): [0.6889523  0.69221362 0.6908646  0.67373573 0.70880914]
Promedio CV Accuracy: 0.6909150761892918
Test Accuracy: 0.6948675496688742


# Modelo de TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Construye vector usando TF-IDF a partir de los datos de entrenamiento
tv = TfidfVectorizer(use_idf=True, min_df=0.0, max_df=1.0)
tv_train_features = tv.fit_transform(train_corpus)
# Construye vector usando TF-IDF a partir de los datos de prueba
tv_test_features = tv.transform(test_corpus)
print("Model TF-IDF:> Entrenamiento:", tv_train_features.shape, " Prueba:", tv_test_features.shape)

Model TF-IDF:> Entrenamiento: (12262, 64183)  Prueba: (6040, 64183)


# Bosque aleatorio con TF-IDF

# Regresión Logística con TF-IDF